# 과제 2: 공공 API를 활용한 데이터 수집 및 분석

**작성자:** [이름]  
**작성일:** 2025-10-23  
**API:** 서울시 따릉이대여소 마스터 정보

---

## 목차
1. [환경 설정](#1-환경-설정)
2. [API 데이터 수집](#2-api-데이터-수집)
3. [ETL 과정](#3-etl-과정)
4. [데이터 시각화](#4-데이터-시각화)
5. [인사이트 도출](#5-인사이트-도출)

---

## 1. 환경 설정

### 1.1 필요한 라이브러리 import

In [ ]:
import os
import json
from datetime import datetime
from dotenv import load_dotenv
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import gaussian_kde
from scipy.spatial.distance import pdist
from collections import Counter
import platform

# 한글 폰트 설정
if platform.system() == 'Windows':
    plt.rc('font', family='Malgun Gothic')
elif platform.system() == 'Darwin':  # macOS
    plt.rc('font', family='AppleGothic')
else:  # Linux
    plt.rc('font', family='NanumGothic')

# 마이너스 기호 깨짐 방지
plt.rc('axes', unicode_minus=False)

print("✅ 라이브러리 import 완료")

### 1.2 환경변수 로드 (API 키)

In [ ]:
# .env 파일에서 API 키 로드
load_dotenv()
API_KEY = os.getenv('SEOUL_API_KEY')

if not API_KEY:
    raise ValueError("❌ API 키가 설정되지 않았습니다. .env 파일을 확인하세요.")
else:
    print(f"✅ API 키 로드 완료: {API_KEY[:10]}...")

---

## 2. API 데이터 수집

### 2.1 API 정보

- **API 명:** 서울시 따릉이대여소 마스터 정보
- **제공기관:** 서울열린데이터광장
- **요청 URL:** `http://openapi.seoul.go.kr:8088/{인증키}/{파일타입}/bikeStationMaster/{시작위치}/{종료위치}/`
- **데이터 형식:** JSON

In [ ]:
# API 요청 설정
BASE_URL = "http://openapi.seoul.go.kr:8088"
SERVICE_NAME = "bikeStationMaster"
FILE_TYPE = "json"
START_INDEX = 1
END_INDEX = 1000

# URL 생성
url = f"{BASE_URL}/{API_KEY}/{FILE_TYPE}/{SERVICE_NAME}/{START_INDEX}/{END_INDEX}/"
print(f"요청 URL: {url}")

### 2.2 데이터 수집 실행

In [ ]:
# API 요청
print("📡 API 요청 중...")

try:
    response = requests.get(url, timeout=10)
    response.raise_for_status()
    data = response.json()
    
    # 응답 확인
    if 'rentBikeStatus' in data:
        result = data['rentBikeStatus']
        stations = result.get('row', [])
        total_count = result.get('list_total_count', 0)
        
        print(f"✅ 데이터 수집 성공!")
        print(f"   총 데이터 건수: {total_count}")
        print(f"   수집된 데이터: {len(stations)}건")
    else:
        print("⚠️ 예상하지 못한 응답 구조")
        print(f"응답 키: {list(data.keys())}")
        
except Exception as e:
    print(f"❌ API 호출 실패: {e}")

### 2.3 수집 데이터 샘플 확인

In [ ]:
# 첫 번째 데이터 샘플 출력
if stations:
    print("📊 첫 번째 대여소 데이터:")
    print(json.dumps(stations[0], ensure_ascii=False, indent=2))

### 2.4 원본 데이터 저장

In [ ]:
# data 디렉토리 생성
os.makedirs('data', exist_ok=True)

# JSON 파일로 저장
with open('data/raw_data.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print("✅ 원본 데이터 저장 완료: data/raw_data.json")

---

## 3. ETL 과정

### 3.1 Extract (추출)

In [ ]:
# JSON 데이터에서 대여소 정보 추출
raw_data = data['rentBikeStatus']['row']
print(f"✅ 추출된 데이터: {len(raw_data)}건")

### 3.2 Transform (변환)

In [ ]:
# DataFrame 생성
df = pd.DataFrame(raw_data)

print("📊 원본 DataFrame 정보:")
print(f"   컬럼: {df.columns.tolist()}")
print(f"   행 수: {len(df)}")
print(f"\n데이터 타입:\n{df.dtypes}")

In [ ]:
# 필요한 컬럼 선택
selected_columns = ['RNTLS_ID', 'ADDR1', 'ADDR2', 'LAT', 'LOT']
df = df[selected_columns].copy()

# 컬럼명 한글로 변경
df.columns = ['대여소ID', '주소1', '주소2', '위도', '경도']

print("✅ 컬럼 선택 및 이름 변경 완료")
print(df.head())

In [ ]:
# 데이터 타입 변환
df['위도'] = pd.to_numeric(df['위도'], errors='coerce')
df['경도'] = pd.to_numeric(df['경도'], errors='coerce')

print("✅ 데이터 타입 변환 완료")
print(df.dtypes)

In [ ]:
# 결측치 확인
print("📋 결측치 확인:")
print(df.isnull().sum())

In [ ]:
# 결측치 제거
initial_count = len(df)
df = df.dropna(subset=['위도', '경도'])
removed_count = initial_count - len(df)

print(f"⚠️ 결측치 제거: {removed_count}건")
print(f"✅ 남은 데이터: {len(df)}건")

In [ ]:
# 이상치 처리 (서울 지역 범위)
initial_count = len(df)
df = df[(df['위도'] >= 37.4) & (df['위도'] <= 37.7)]
df = df[(df['경도'] >= 126.7) & (df['경도'] <= 127.2)]
removed_count = initial_count - len(df)

print(f"📍 서울 지역 범위 필터링")
print(f"   위도: 37.4 ~ 37.7")
print(f"   경도: 126.7 ~ 127.2")
print(f"⚠️ 범위 밖 데이터 제거: {removed_count}건")
print(f"✅ 최종 데이터: {len(df)}건")

In [ ]:
# 구 정보 추출
df['구'] = df['주소1'].str.extract(r'(.*?구)')[0]

print("✅ 구 정보 추출 완료")
print(f"\n구 목록:\n{df['구'].value_counts()}")

### 3.3 Load (적재)

In [ ]:
# CSV 파일로 저장
csv_path = 'data/seoul_bike_stations.csv'
df.to_csv(csv_path, index=False, encoding='utf-8-sig')

print(f"✅ CSV 파일 저장 완료: {csv_path}")
print(f"\n최종 데이터 샘플:")
print(df.head(10))

---

## 4. 데이터 시각화

### 4.1 시각화 디렉토리 생성

In [ ]:
os.makedirs('visualizations', exist_ok=True)
print("✅ 시각화 디렉토리 생성 완료")

### 4.2 시각화 1: 대여소 지리적 분포

In [ ]:
plt.figure(figsize=(14, 10))
plt.scatter(df['경도'], df['위도'], alpha=0.5, s=50, c='blue', edgecolors='black', linewidth=0.5)
plt.title('서울시 따릉이 대여소 지리적 분포', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('경도 (Longitude)', fontsize=12)
plt.ylabel('위도 (Latitude)', fontsize=12)
plt.grid(True, alpha=0.3, linestyle='--')

plt.text(0.02, 0.98, f'총 대여소: {len(df)}개',
         transform=plt.gca().transAxes,
         fontsize=12, verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.7))

plt.tight_layout()
plt.savefig('visualizations/station_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ 시각화 1 저장 완료")

### 4.3 시각화 2: 구별 대여소 분포

In [ ]:
district_count = df['구'].value_counts().sort_values(ascending=False)

plt.figure(figsize=(14, 8))
bars = plt.bar(range(len(district_count)), district_count.values,
              color='steelblue', edgecolor='black', alpha=0.7, linewidth=1.5)
plt.xticks(range(len(district_count)), district_count.index, rotation=45, ha='right', fontsize=10)
plt.title('서울시 구별 따릉이 대여소 분포', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('구', fontsize=12)
plt.ylabel('대여소 수', fontsize=12)
plt.grid(axis='y', alpha=0.3, linestyle='--')

for i, bar in enumerate(bars):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}', ha='center', va='bottom', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.savefig('visualizations/district_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ 시각화 2 저장 완료")

### 4.4 시각화 3: 밀집도 히트맵

In [ ]:
plt.figure(figsize=(14, 10))

xy = np.vstack([df['경도'], df['위도']])
z = gaussian_kde(xy)(xy)

scatter = plt.scatter(df['경도'], df['위도'], c=z, s=50,
                     cmap='YlOrRd', alpha=0.6, edgecolors='black', linewidth=0.5)
plt.colorbar(scatter, label='밀집도')
plt.title('서울시 따릉이 대여소 밀집도 히트맵', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('경도 (Longitude)', fontsize=12)
plt.ylabel('위도 (Latitude)', fontsize=12)
plt.grid(True, alpha=0.3, linestyle='--')

plt.tight_layout()
plt.savefig('visualizations/density_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ 시각화 3 저장 완료")

---

## 5. 인사이트 도출

### 5.1 전체 통계

In [ ]:
print("=" * 60)
print("📊 서울시 따릉이 대여소 분석 인사이트")
print("=" * 60)

print(f"\n1️⃣ 전체 대여소 현황")
print(f"   - 총 대여소 수: {len(df)}개")
print(f"   - 위도 범위: {df['위도'].min():.4f} ~ {df['위도'].max():.4f}")
print(f"   - 경도 범위: {df['경도'].min():.4f} ~ {df['경도'].max():.4f}")

### 5.2 구별 분포 분석

In [ ]:
print(f"\n2️⃣ 구별 대여소 분포 (TOP 5)")
district_count = df['구'].value_counts().head()
for i, (district, count) in enumerate(district_count.items(), 1):
    print(f"   {i}. {district}: {count}개 ({count/len(df)*100:.1f}%)")

print(f"\n   ➡️ {district_count.index[0]}이 가장 많은 대여소를 보유하고 있습니다.")
print(f"   ➡️ 상위 5개 구가 전체의 {district_count.sum()/len(df)*100:.1f}%를 차지합니다.")

### 5.3 대여소 간 평균 거리

In [ ]:
coordinates = df[['경도', '위도']].values
distances = pdist(coordinates, metric='euclidean')
avg_distance = distances.mean()

print(f"\n3️⃣ 대여소 간 평균 거리")
print(f"   - {avg_distance:.4f}도 (약 {avg_distance*111:.2f}km)")

### 5.4 주요 인사이트

#### 인사이트 1: 지역별 대여소 분포 불균형
- 강남구, 영등포구, 마포구 등 주요 상업지역에 대여소가 집중
- 도심 지역(중구, 종로구)의 대여소 밀집도가 높음
- 외곽 지역의 대여소 접근성 개선 필요

#### 인사이트 2: 교통 허브 중심의 배치 전략
- 지하철역 주변에 대여소가 집중 배치되어 있음
- 출퇴근 및 환승 수요를 고려한 전략적 배치
- 마지막 1마일(Last Mile) 교통수단으로 활용

#### 인사이트 3: 공원 및 관광지 주변 인프라
- 한강공원, 올림픽공원 등 주요 공원 주변에 대여소 밀집
- 여가 및 레저 활동을 위한 자전거 이용 수요 반영
- 주말과 평일의 이용 패턴 차이 예상

#### 인사이트 4: 대여소 간 적정 거리 유지
- 대부분의 대여소가 300~500m 간격으로 배치
- 도보 5~10분 거리에서 접근 가능한 편의성 확보
- 서비스 품질 유지를 위한 체계적 인프라 구축

---

## 결론

서울시 따릉이 대여소 마스터 정보 API를 활용하여 대여소의 지리적 분포와 특성을 분석했습니다.

### 과제 수행 요약

1. **API 데이터 수집**: 서울시 열린데이터 광장 API를 통해 100건 이상의 대여소 정보 수집
2. **ETL 과정**: Extract(추출) → Transform(변환/정제) → Load(저장) 과정 수행
3. **데이터 시각화**: 3가지 시각화(지리적 분포, 구별 분포, 밀집도 히트맵) 생성
4. **인사이트 도출**: 4가지 주요 인사이트 발견 및 분석

### 활용 기술
- **Python**: requests, pandas, matplotlib, seaborn, scipy
- **데이터 처리**: JSON 파싱, DataFrame 변환, 이상치 제거
- **시각화**: 산점도, 막대 그래프, 히트맵
- **통계 분석**: KDE(Kernel Density Estimation), 거리 계산

---

**작성 완료일:** 2025-10-23